In [15]:
import imdb
import requests
from bs4 import BeautifulSoup
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

ia = imdb.IMDb()

In [2]:
# Otrzymanie comentarzów 
def get_imdb_reviews(movie_title):
    url = f"https://www.imdb.com/title/{movie_title}/reviews"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    review_containers = soup.find_all("div", class_="lister-item-content")

    reviews = []
    for container in review_containers:
        review_text = container.find("div", class_="text").text.strip()
        reviews.append(review_text)

    return reviews

In [3]:
def load_data():
    movie_ids = ['tt1375666']  # Id filmu dla treningu
    reviews = []
    for movie_id in movie_ids:
        reviews.extend(get_imdb_reviews(movie_id))
    return reviews

In [4]:
reviews = load_data()
# ocena comentarza (0 - zly 1 - dobry)
labels = np.array([1, 0, 1, 1, 0, 1, 1, 0, 0, 0,
                   0, 1, 0, 1, 0, 0, 1, 1, 0, 1,
                   1, 0, 1, 0, 1])

In [5]:
# Tworzenie i trening modelu
tokenizer = Tokenizer(num_words=10000)  # Ograniczenie słownictwa do 10 000 najczęściej używanych słów.
tokenizer.fit_on_texts(reviews)
sequences = tokenizer.texts_to_sequences(reviews)
padded_sequences = pad_sequences(sequences, maxlen=100)  # Ogranicz długość sekwencji do 100 słów



In [6]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(10000, 16, input_length=100),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

2023-05-26 15:52:23.696529: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-26 15:52:24.009555: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-26 15:52:24.009854: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-26 15:52:24.019175: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-26 15:52:24.019287: I tensorflow/compile

In [7]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(padded_sequences, labels, epochs=2)

Epoch 1/2


2023-05-26 15:52:33.058295: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7f6b4c016500 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-05-26 15:52:33.058426: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce MX150, Compute Capability 6.1
2023-05-26 15:52:33.170474: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-05-26 15:52:34.478166: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-05-26 15:52:35.999336: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-05-26 15:52:36.413497: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1/1 [==============================] - 9s 9s/step - loss: 0.6928 - accuracy: 0.6000
Epoch 2/2
1/1 [==============================] - 0s 94ms/step - loss: 0.6919 - accuracy: 0.7600


In [8]:
# Funkcja analizy filmu
def analyze_movie(movie_id):
    reviews = get_imdb_reviews(movie_id)
    sequences = tokenizer.texts_to_sequences(reviews)
    padded_sequences = pad_sequences(sequences, maxlen=100)
    predictions = model.predict(padded_sequences)
    average_sentiment = predictions.mean()
    return average_sentiment

In [17]:
def get_imdb_movie_id(movie_title):
    movies = imdb.search_movie(movie_title)
    movie = movies[0]
    movie_id = movie.getID()
    return 'tt' + movie_id

In [18]:
title = 'Toy Story'
mv = get_imdb_movie_id(title)
#movie_title = get_imdb_movie_title_by_id(movie_id)
sentiment = analyze_movie(mv)
if sentiment > 0.5:
    print(f"The movie '{title}' is recommended.")
else:
    print(f"The movie '{title}' is not recommended.")

AttributeError: module 'imdb' has no attribute 'search_movie'